Here we will compare different samplig methods for the target `donut_target`, specifically the ess method 

### Importing the needed libraries 

In [1]:
from cuqi.distribution import DistributionGallery, Gaussian, JointDistribution
from cuqi.testproblem import Poisson1D
from cuqi.problem import BayesianProblem
import cuqi
import inspect
import numpy as np
import matplotlib.pyplot as plt
from cuqi.sampler import MH, CWMH, ULA, MALA, NUTS
import time
import scipy.stats as sps
from scipy.stats import gaussian_kde
import pandas as pd
import cProfile, pstats, io
from pstats import SortKey
import TableAutomization as TA

### The donut distribution


Out of all of the sampling methods we choose 5:
- MH with fixed samlpler
- MH with adapted sampler
- ULA
- MALA 
- NUTSA

We create a general sampling function, that given certain inputs will compute the according MCMC sampling

In [2]:
# The donut distribution 
target_donut = DistributionGallery("donut")


In [4]:
TA.create_table(target_donut, 0.05, 1000,2, x0 = np.array([0,0]), seed = 12 )

Sample 1002 / 1002

Average acceptance rate: 0.851 

Sample 1002 / 1002

Average acceptance rate: 0.574 MCMC scale: 0.33174280419572777 

Sample 1002 / 1002
Sample 1002 / 1002
Sample 1002 / 1002


,Sampling Method,No. of Samples,No. of Burn-ins,Scaling Factor,ESS (v0),ESS (v1),AR,LogPDF
0,MH_fixed,1000,2,0.05,5.210,1.445,0.851,1002.0
1,MH_adapted,1000,2,0.05,1.809,7.280,0.574,1002.0
2,ULA,1000,2,0.05,2.412,1.944,1.000,1002.0
3,MALA,1000,2,0.05,1000.000,1000.000,0.000,1002.0
4,NUTS,1000,2,0.05,252.605,316.402,0.961,0.0
